In [1]:
# !pip3 install torchtext
import time
import math
import os
import string


import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

from copy import deepcopy
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from torchtext.data.utils import get_tokenizer
from tqdm import tqdm

from src.dataset import HeadlineDataset, collate_fn, generate_vocab_map
from src.models import LSTMClassificationModel
from src.text_cleaning import clean_text





In [2]:
tokenizer = get_tokenizer("basic_english")
data = pd.read_csv("data/data_all.csv")
data["tokenized"] = data["text"].apply(clean_text).apply(tokenizer)
data.head()
print(len(data))




18724


In [3]:
location_less = data['location'].isnull().sum()
print(location_less)
text_less = 0
for i in range(len(data)):
    text = data['tokenized'][i]
    if (len(text) < 3):
        text_less += 1
    if data['location'] is None:
        location_less += 1
print(text_less)

5862
141


In [4]:
import numpy as np
from scipy import stats

token_length = []
for i in range(len(data)):
    text = data['tokenized'][i]
    token_length.append(len(text))
token_length = np.array(token_length)
mean_length = np.mean(token_length)
mode_length = stats.mode(token_length)
median_length = np.median(token_length)
variance = np.sqrt(np.var(token_length))
print(mean_length,mode_length, median_length, variance)

16.08128604998932 ModeResult(mode=array([20]), count=array([1394])) 17.0 5.737168059418903


In [5]:
train_val_data, test_data = train_test_split(data, train_size=0.9, test_size=0.1, shuffle=False)
print(train_val_data.head())
print(type(train_val_data))

   id    keyword  location                                               text  \
0   0   attacked     India  Largscale violence in #Bhainsa, dist Nirmal of...   
1   1    trouble       NaN  Hi there. I'm sorry to hear that you are havin...   
2   2     derail       NaN  Design of AR-15 could derail charges tied to p...   
3   3  emergency  The Void  I've never posed in the office block coz whene...   
4   4   hostages       NaN  “We are not citizens. We never were. We are ca...   

   label                                          tokenized  
0      0  [largscale, violence, in, bhainsa, dist, nirma...  
1      1  [hi, there, im, sorry, to, hear, that, you, ar...  
2      0  [design, of, arnumber, could, derail, charges,...  
3      0  [ive, never, posed, in, the, office, block, co...  
4      0  [we, are, not, citizens, we, never, were, we, ...  
<class 'pandas.core.frame.DataFrame'>


In [6]:
# !pip3 install nltk
import nltk
from nltk.stem import PorterStemmer
nltk.download('stopwords')


ps=PorterStemmer()

def stemming(txt):
    words=[ps.stem(word) for word in txt]
    return words

stopwords=nltk.corpus.stopwords.words('english')

def remove_stopwords(tokenize):
    clean=[word for word in tokenize if word not in stopwords]
    return clean

train_val_data['tokenized_after_clean']=train_val_data['tokenized'].apply(lambda x: remove_stopwords(x))
test_data['tokenized_after_clean']=test_data['tokenized'].apply(lambda x: remove_stopwords(x))

train_val_data['tokenized_after_clean']=train_val_data['tokenized'].apply(lambda x: stemming(x))
test_data['tokenized_after_clean']=test_data['tokenized'].apply(lambda x: stemming(x))


print(train_val_data.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/songmingliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


   id    keyword  location                                               text  \
0   0   attacked     India  Largscale violence in #Bhainsa, dist Nirmal of...   
1   1    trouble       NaN  Hi there. I'm sorry to hear that you are havin...   
2   2     derail       NaN  Design of AR-15 could derail charges tied to p...   
3   3  emergency  The Void  I've never posed in the office block coz whene...   
4   4   hostages       NaN  “We are not citizens. We never were. We are ca...   

   label                                          tokenized  \
0      0  [largscale, violence, in, bhainsa, dist, nirma...   
1      1  [hi, there, im, sorry, to, hear, that, you, ar...   
2      0  [design, of, arnumber, could, derail, charges,...   
3      0  [ive, never, posed, in, the, office, block, co...   
4      0  [we, are, not, citizens, we, never, were, we, ...   

                               tokenized_after_clean  
0  [largscal, violenc, in, bhainsa, dist, nirmal,...  
1  [hi, there, im, sorri

In [7]:
ngram_range = (2,3)
max_features = 5000

In [97]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(train_val_data['label'])
Test_Y = Encoder.fit_transform(test_data['label'])

Train_X = train_val_data['tokenized_after_clean']
Test_X = test_data['tokenized_after_clean']

# # ---------TfidfVectorizer---------
# Tfidf_vect = TfidfVectorizer(max_features=5000, preprocessor=' '.join, stop_words='english')
# Tfidf_vect.fit(Train_X)
# Train_X_Tfidf = Tfidf_vect.transform(Train_X)
# Test_X_Tfidf = Tfidf_vect.transform(Test_X)

# ---------BoW---------
Tfidf_vect = CountVectorizer(preprocessor=''.join, stop_words='english', ngram_range=ngram_range, max_features=max_features)
Tfidf_vect.fit(Train_X)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)


# -----------SMOTE-------------
# !pip install -U imbalanced-learn

# from imblearn.over_sampling import SMOTE 
# sm = SMOTE(random_state=42)
# Train_X_Tfidf, Train_Y = sm.fit_resample(Train_X_Tfidf, Train_Y)
# Test_X_Tfidf, Test_Y = sm.fit_resample(Test_X_Tfidf, Test_Y)



In [98]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)

predictions_NB = Naive.predict(Test_X_Tfidf)

print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
print("Naive Bayes f-1 Score -> ",f1_score(predictions_NB, Test_Y))

confusion_matrix(predictions_NB, Test_Y)

Naive Bayes Accuracy Score ->  50.0
Naive Bayes f-1 Score ->  0.0


array([[1319, 1319],
       [   0,    0]])

In [99]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


SVM = svm.SVC(C=10.5, kernel='linear', gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)


predictions_SVM = SVM.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM f-1 Score -> ",f1_score(predictions_SVM, Test_Y))
confusion_matrix(predictions_SVM, Test_Y)

SVM Accuracy Score ->  50.0
SVM f-1 Score ->  0.6666666666666666


array([[   0,    0],
       [1319, 1319]])

In [83]:
SVM = svm.SVC(kernel='rbf', C=7.5)
SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM f-1 Score -> ",f1_score(predictions_SVM, Test_Y))
confusion_matrix(predictions_SVM, Test_Y)

SVM Accuracy Score ->  84.6095526914329
SVM f-1 Score ->  0.830550918196995


array([[1237,  324],
       [  82,  995]])

In [84]:
SVM = svm.SVC(C=7.5, kernel='sigmoid', gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM f-1 Score -> ",f1_score(predictions_SVM, Test_Y))
print(np.sum(np.array(predictions_SVM)))

SVM Accuracy Score ->  65.5420773313116
SVM f-1 Score ->  0.5172596919808815
564


In [86]:
SVM = svm.SVC(C=9.0, kernel='poly', gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM f-1 Score -> ",f1_score(predictions_SVM, Test_Y))
confusion_matrix(predictions_SVM, Test_Y)

SVM Accuracy Score ->  54.51099317664898
SVM f-1 Score ->  0.16550764951321278


array([[1319, 1200],
       [   0,  119]])

In [96]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression(random_state=0, C=1.1)
log.fit(Train_X_Tfidf,Train_Y)

predictions_log = log.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(predictions_log, Test_Y)*100)
print("SVM f-1 Score -> ",f1_score(predictions_log, Test_Y))
confusion_matrix(predictions_SVM, Test_Y)

SVM Accuracy Score ->  80.6292645943897
SVM f-1 Score ->  0.8026264967168791


array([[1319, 1200],
       [   0,  119]])

In [47]:
base_estimator = naive_bayes.MultinomialNB()
# base_estimator = svm.SVC(kernel='rbf', C=7.5)

In [87]:
from sklearn.ensemble import BaggingClassifier

bc = BaggingClassifier(base_estimator=base_estimator, n_estimators=800)
bc.fit(Train_X_Tfidf, Train_Y)

predictions_bc = bc.predict(Test_X_Tfidf)
print("Bagging  Accuracy Score -> ",accuracy_score(predictions_bc, Test_Y)*100)
print("Bagging regression f-1 Score -> ",f1_score(predictions_bc, Test_Y))


Bagging  Accuracy Score ->  80.17437452615617
Bagging regression f-1 Score ->  0.7968932038834952


In [88]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(base_estimator=None, n_estimators=800)
abc.fit(Train_X_Tfidf, Train_Y)

predictions_abc = abc.predict(Test_X_Tfidf)
print("Bagging  Accuracy Score -> ",accuracy_score(predictions_abc, Test_Y)*100)
print("Bagging regression f-1 Score -> ",f1_score(predictions_abc, Test_Y))


Bagging  Accuracy Score ->  82.48673237300986
Bagging regression f-1 Score ->  0.8206521739130436


In [89]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=800)
gbc.fit(Train_X_Tfidf, Train_Y)

predictions_gbc = gbc.predict(Test_X_Tfidf)
print("Bagging  Accuracy Score -> ",accuracy_score(predictions_gbc, Test_Y)*100)
print("Bagging regression f-1 Score -> ",f1_score(predictions_gbc, Test_Y))


Bagging  Accuracy Score ->  84.64746019711903
Bagging regression f-1 Score ->  0.8378053664397277


In [90]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=550, criterion='gini')
rf.fit(Train_X_Tfidf, Train_Y)

predictions_rf = rf.predict(Test_X_Tfidf)
print("Bagging  Accuracy Score -> ",accuracy_score(predictions_rf, Test_Y)*100)
print("Bagging regression f-1 Score -> ",f1_score(predictions_rf, Test_Y))

Bagging  Accuracy Score ->  83.51023502653526
Bagging regression f-1 Score ->  0.8262085497403117


In [ ]:
# !pip3 install lightgbm
import lightgbm
from sklearn import metrics


fit = lightgbm.Dataset(Train_X_Tfidf, Train_Y)
val = lightgbm.Dataset(Test_X_Tfidf, Test_Y, reference=fit)

model = lightgbm.train(
    params={
        'learning_rate': 0.01,
        'objective': 'binary'
    },
    train_set=fit,
    num_boost_round=10000,
    valid_sets=(fit, val),
    valid_names=('fit', 'val'),
    early_stopping_rounds=20,
    verbose_eval=100
)

y_pred = model.predict(X_test)

print()
print(f"Test's ROC AUC: {metrics.roc_auc_score(y_test, y_pred):.5f}")
print(f"Test's logloss: {metrics.log_loss(y_test, y_pred):.5f}")